In [ ]:
import requests
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import MetaTrader5 as mt5
from twelvedata import TDClient 
import time


class ForexModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(ForexModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_sizes[2], output_size),
            nn.Tanh()  # Constrains the output to [-1, 1]
        )

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.fc(x[:, -1, :])  # Take the last time step's output
        return x

features = ['open', 'high', 'low', 'close', 'sma1', 'sma2', 'sma3', 'sma4', 'rsi']

input_size = len(features)
hidden_sizes = [128, 64, 32]  # Descending sizes for LSTM layers
output_size = 1

MODEL_PATH = "./Model/3layer_lstm.v.6.model"
model = ForexModel(input_size, hidden_sizes, output_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(MODEL_PATH, map_location=device, weights_only=False))
model.to(device)
model.eval()

scaler = MinMaxScaler()
API_KEY = 'e6b3ac6147a04e34bca8d5f1011b728c'
td = TDClient(apikey=API_KEY)
def fetch_live_data(symbol, lookback, sequence_length):
    
    ts = td.time_series(symbol="EUR/USD", interval=f'{time_frame}min', outputsize=sequence_length).with_sma(time_period=200).with_sma(time_period=100).with_sma(time_period=50).with_sma(time_period=13).with_rsi(time_period=14).as_pandas()
    
    # print(ts.head())
    return ts

def prepare_features(data):
    data[features] = scaler.fit_transform(data[features].values)
    sequence = data[features].values[-sequence_length:]
    return torch.tensor(sequence, dtype=torch.float32).unsqueeze(0)

def open_trade(symbol, action, lot, sl, tp):
    price = mt5.symbol_info_tick(symbol).ask if action == "buy" else mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY if action == "buy" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": 234000,
        # "comment": f"AI Forex Trade at {datetime.now()}",
        # "type_time": mt5.ORDER_TIME_GTC,
        # "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    # print(result)
    if result is None:
        print("Failed to send order:", mt5.last_error())
        return
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(f"Order failed: {result.retcode}")
    else:
        print("Trade executed: ", action)

# Live prediction and trading loop
def live_trading():
    if not mt5.initialize():
        print("Failed to initialize MT5:", mt5.last_error())
        return

    if not mt5.login(20037567, '1380_EskI', "WMMarkets-Demo"):
        print(f"Failed to log in to account {20037567}")
        print(f"Error code: {mt5.last_error()}")
        mt5.shutdown()
        
    # account_info = mt5.account_info()
    # print(account_info)
    
    while True:
        live_data = fetch_live_data(symbol, lookback, sequence_length)
        if live_data is not None:
            features_tensor = prepare_features(live_data).to(device)

            with torch.no_grad():
                prediction = model(features_tensor).item()

            print(f"Prediction: {prediction} | @ {datetime.now().strftime("%H:%M:%S")}")

            price = mt5.symbol_info_tick(symbol).ask if prediction > 0 else mt5.symbol_info_tick(symbol).bid
            
            trade = False
            
            if prediction > thereshold+0.02:
                # sl = price - prediction*sl_pips 
                # tp = price + prediction*tp_pips 
                sl = price - sl_pips 
                tp = price + tp_pips 
                action = "buy" 
                trade = True
                
            elif prediction < thereshold-0.02:
                # sl = price + prediction*sl_pips
                # tp = price - prediction*tp_pips
                sl = price + sl_pips
                tp = price - tp_pips
                action = "sell"
                trade = True
            
            else:
                print("No Trade executed, Prediction within threshold.")
            
            
            
            if mt5.positions_total() < max_open_positions and trade:
                open_trade(symbol, action, lot, sl, tp)
            else:
                if trade:
                    print(f"No Trade executed ({action}), Maximum open positions reached.")
                else:
                    print("No Trade executed, Maximum open positions reached.")
        else:
            print("No data available for trading.")
            

        print()
        time.sleep(wait*60)

# Parameters
symbol = "EURUSD"
lot = 0.1
thereshold = 0.30
sl_pips = 0.00025
tp_pips = 0.00050
lookback = 50
sequence_length:int = 10
max_open_positions:int = 999999999999999999999
time_frame:int = 1
wait:float = 15 + .1

live_trading()


Prediction: 0.453352153301239 | @ 13:54:50
Order failed: 10016

